In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# #TODO1: EDA


In [ ]:
#Profiling Pandas
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
# Overview about data
data_df = pd.DataFrame(np.random.rand(100, 8), columns=["Email", "Address", "Avatar", "Avg. Session Length", "Time on App","Time on Website","Length of Membership", "Yearly Amount Spent"])
profile = ProfileReport(data_df, title="Ecommerce Customers Report",explorative=True)

In [ ]:
profile.to_notebook_iframe()

In [ ]:
data_df=pd.read_csv('/kaggle/input/ecommerce-customers/Ecommerce Customers.csv')
data_df.head()

# Data Manipulation

In [ ]:
# rename cols for ease of manipulate
data_df.columns = [re.sub("\s", "_", c.lower()) for c in data_df.columns]
data_df.head()

In [ ]:
# check additional information we can get from address
data_df['address'][0].splitlines()[1].split()

In [ ]:
[re.findall('[A-Z][^A-Z]*', a) for a in data_df.avatar]

In [ ]:
# Add field zip_code, state domain and avatar theme
data_df['zip_code'] = data_df['address'].map(lambda x: x.splitlines()[1].split()[-2])
data_df['state'] = data_df['address'].map(lambda x: x.splitlines()[1].split()[-1])
data_df['avatar_theme'] = [re.findall('[A-Z][^A-Z]*', a)[-1] for a in data_df.avatar] 

In [ ]:
data_df.head()

In [ ]:
# Distribution of single value - zip_code & avatar_theme
fig,axes = plt.subplots(nrows=2,ncols=1,dpi=299,figsize = (16,8))

data_df.zip_code.value_counts().plot.bar(ax=axes[0], title="Distribution of zip_code value")
data_df.avatar_theme.value_counts().plot.bar(ax=axes[1], title="Distribution of state value")

In [ ]:
sns.pairplot(data_df)

In [ ]:
sns.heatmap(data_df.corr(), annot=True)

Vài inside từ biến Yearly Amount Spent tren Profile:
- Yearly amount spend có mối quan hệ mật thiết với Time on Website, Avg. Session Length và Length of Membership

In [ ]:
# Scatter plot w/ regression line showing the relationship of Yearly Amount Spent wwith Time on Website, Avg. Session Length và Length of Membership
fig,axes = plt.subplots(nrows=1,ncols=3,dpi=120,figsize = (16,4))

plot0=sns.regplot(x=data_df['avg._session_length'],y=data_df['yearly_amount_spent'], ax=axes[0], color='green')
plot1=sns.regplot(x=data_df['time_on_website'],y=data_df['yearly_amount_spent'], ax=axes[1], color='green')
plot1=sns.regplot(x=data_df['length_of_membership'],y=data_df['yearly_amount_spent'], ax=axes[2], color='green')
plt.tight_layout()

In [ ]:
data_df.groupby('avatar_theme').yearly_amount_spent.median().sort_values()

In [ ]:
data_df.groupby('avatar_theme').yearly_amount_spent.sum().sort_values()

-> Nhận xét: 
- Từ biểu đồ trên có thể thấy, nếu user có thời gian là membership của app càng tăng thì tổng số lượng giao dịch trên apps càng nhiều. Ngoài ra trung bình thời gian sử dụng apps càng nhiều thì tổng số lượng tiền giao dịch trên apps có tăng nhưng không đáng kể -> điều này chứng tỏ khách hàng có độ chần chừ nhất định khi giao dịch.
- Màu sắc avatar mặc dù màu xanh và trắng chiếm số đông màu trên tổng data và trên tổng top users nộp tiền vào nhiều nhưng tính trên mức độ trung bình thì những màu này không phải là top.

# Linear Regressions
Thực hiện bài toán Linear Regression với biến target (y) là Yearly Amount Spent. Lưu ý trình bày đầy đủ các bước:
- Data Processing
- Split Train & Test Data
- Modeling
- Evaluation

In [ ]:
!pip install -U scikit-learn

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
yearly_count = data_df['yearly_amount_spent']
length_membership = data_df['length_of_membership']

x = np.array(length_membership).reshape(-1, 1)
y = np.array(yearly_count)

#Splitting the data into Train and Test
from sklearn.model_selection import train_test_split 
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=1/3, random_state=0)

#Fitting simple linear regression to the Training Set
from sklearn.linear_model import LinearRegression 
regressor = LinearRegression()
regressor.fit(xtrain, ytrain)

#Predicting the yearly amount spent
pred = regressor.predict(xtest)

#Visualizing the training Test Results 
plt.scatter(xtrain, ytrain, color= 'red')
plt.plot(xtrain, regressor.predict(xtrain), color = 'blue')
plt.title ("Visuals for Training Dataset")
plt.xlabel("length of membership")
plt.ylabel("yearly count spent")
plt.show()

#Visualizing the Test Results 
plt.scatter(xtest, ytest, color= 'red')
plt.plot(xtrain, regressor.predict(xtrain), color = 'blue')
plt.title ("Visuals for Test Dataset")
plt.xlabel("length of membership")
plt.ylabel("yearly count spent")
plt.show()

# Customer Clustering
- Áp dụng phương pháp Clustering lên bộ data để chia customers thành n nhóm (n tuỳ theo bạn lựa chọn)
- Trên n nhóm đã phân ra bằng clustering, trình bày chỉ số thông kê và phân phối của các biến sau: Time on App, Avg. Session Length, Time on Website, Length of Membership
- Visualize plot các yếu tố trên theo cặp. Ví dụ như bên dưới (cluster = các nhóm khác hàng đã chia) 

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
# Create cluster feature
kmeans = KMeans(n_clusters=5)
X = data_df.loc[:,["time_on_app", "time_on_website", "length_of_membership","yearly_amount_spent"]]
X["Cluster"] = kmeans.fit_predict(X)
X["Cluster"] = X["Cluster"].astype("category")

X.head()

In [ ]:
sns.pairplot(data=X,hue="Cluster")

Nhận xét:
Với các biểu đồ trên thì ta nên chú trọng vào khách hàng số 1 bằng cách:
- Có nhiều chương trình trò chơi khuyến khích người dùng dùng apps
- Nhiều chương trình khuyến mại khi thanh toán trên apps
- Giảm giá khi người dùng gia hạn membership hoặc chương trình khuyến mãi khi đăng kí membership trong một khoảng thời gian nhất định